In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import os
import shutil
from lxml import etree
import cv2

In [ ]:
dataset_path = r"E:\ANPR\License Dataset"

In [ ]:
width = np.array([])
height = np.array([])

for imgName in tqdm(os.listdir(os.path.join(dataset_path, "images"))):
    img = cv2.imread(os.path.join(dataset_path, "images", imgName))
    height = np.append(height, img.shape[0])
    width = np.append(width, img.shape[1])

In [ ]:
fig = plt.figure(figsize=(20, 5))

fig.add_subplot(1, 2, 1)
sns.barplot(height, np.arange(0, len(height), 1))
plt.axis('off')
plt.title("Height")

fig.add_subplot(1, 2, 2)
sns.barplot(width, np.arange(0, len(width), 1))
plt.axis('off')
plt.title("Width")

plt.show()

In [ ]:
print("---------- HEIGHT ----------")
print(f"Mean : {height.mean():.2f}")
print(f"Variance : {height.var():.2f}")
print(f"Standard Diviation : {height.std():.2f}")
print(f"Max : {height.max():.2f}")
print(f"Min : {height.min():.2f}")

print("\n---------- WIDTH  ----------")
print(f"Mean : {width.mean():.2f}")
print(f"Variance : {width.var():.2f}")
print(f"Standard Diviation : {width.std():.2f}")
print(f"Max : {width.max():.2f}")
print(f"Min : {width.min():.2f}")

In [ ]:
# Resizing
IMAGE_SIZE = 350

for imgName in tqdm(os.listdir(os.path.join(dataset_path, "images"))):
    path = os.path.join(dataset_path, "images", imgName)
    img = cv2.imread(path)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    os.remove(path)
    cv2.imwrite(path, img)

In [ ]:
imgPath = []
for imgName in tqdm(sorted(os.listdir(os.path.join(dataset_path, "images")))):
    imgPath.append(imgName)

coors = []
for annot in tqdm(sorted(os.listdir(os.path.join(dataset_path, "annotations")))):
    tree = etree.parse(os.path.join(dataset_path, "annotations", annot))

    for dim in tree.xpath("size"):
        width = int(dim.xpath("width")[0].text)
        height = int(dim.xpath("height")[0].text)
    
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text) / (width / IMAGE_SIZE)
        ymin = int(dim.xpath("ymin")[0].text) / (height / IMAGE_SIZE)
        xmax = int(dim.xpath("xmax")[0].text) / (width / IMAGE_SIZE)
        ymax = int(dim.xpath("ymax")[0].text) / (height / IMAGE_SIZE)
    coors.append([int(xmax), int(ymax), int(xmin), int(ymin)])

In [ ]:
coors = np.array(coors)
data = pd.DataFrame({"name": imgPath, "xmax": coors[:, 0], "ymax": coors[:, 1], "xmin": coors[:, 2], "ymin": coors[:, 3]})
data.head()

In [ ]:
fig = plt.figure(figsize=(18, 8))
rows, columns = 2, 3

for i in range(1, rows*columns+1):
    fig.add_subplot(rows, columns, i)

    pos = np.random.randint(0, len(data) + 1)
    img = cv2.imread(os.path.join(dataset_path, data.loc[pos, "name"]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.rectangle(img, (data.loc[pos,"xmax"], data.loc[pos,"ymax"]),
                        (data.loc[pos,"xmin"], data.loc[pos,"ymin"]), (255, 0, 0), 2)

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image {i}")

plt.show()

## Augmentation

### Augment Class

In [ ]:
class Augmentation():
    def __init__(self, data):
        
        self.df = data
        self.image_path_src = r"E:\ANPR\License Dataset\images"
        self.image_path_des = r"E:\ANPR\License Dataset\augmented images"
        self.augment = pd.DataFrame(columns = ["name", "xmax", "ymax", "xmin", "ymin"])
    
    def flip_H(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Horizontal Flip Image
                img = cv2.flip(img, 1)

                # Horizontal Flip Mask
                mask = cv2.flip(mask, 1)

                cols, rows = np.nonzero(mask[:, :, 0])    
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_H.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_H.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def flip_V(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Horizontal Flip Image
                img = cv2.flip(img, 0)

                # Horizontal Flip Mask
                mask = cv2.flip(mask, 0)

                cols, rows = np.nonzero(mask[:, :, 0])    
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_V.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_V.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def rotate(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Rotation
                theta = np.random.randint(-20, 20)

                ## Image
                img = ndimage.rotate(img, angle=theta)
                img = cv2.resize(img, (350, 350), interpolation=cv2.INTER_AREA)

                ## Mask
                mask = ndimage.rotate(mask, angle=theta)
                mask = cv2.resize(mask, (350, 350), interpolation=cv2.INTER_AREA)

                cols, rows = np.nonzero(mask[:, :, 0])
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_R.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_R.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def shear(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Shearing
                angle = np.random.uniform(-0.3, 0.3, 1)[0]

                ## Image
                afine_tf = tf.AffineTransform(shear=angle)
                img = tf.warp(img, inverse_map=afine_tf)
                img = cv2.convertScaleAbs(img, alpha=(255.0))

                ## Mask
                afine_tf = tf.AffineTransform(shear=angle)
                mask = tf.warp(mask, inverse_map=afine_tf)

                cols, rows = np.nonzero(mask[:, :, 0])    
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_S.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_S.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def brightness(self):
        
        transform = A.Compose([ A.RandomBrightness(p=1, limit=[-0.3, 0.3]) ])
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Brightness (Image Only)
                augmentations = transform(image=img)
                img = augmentations["image"]

                cols, rows = np.nonzero(mask[:, :, 0])
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_B.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_B.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def width_shift(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Horizontal Width Shift
                percent_shift = np.random.uniform(-0.05, 0.05, 1)[0]

                ## Image
                T = np.float32([[1, 0, percent_shift*img.shape[0]], [0, 1, 0]])
                img = cv2.warpAffine(img, T, (img.shape[0], img.shape[1]))

                ## Mask
                T = np.float32([[1, 0, percent_shift*mask.shape[0]], [0, 1, 0]])
                mask = cv2.warpAffine(mask, T, (mask.shape[0], mask.shape[1]))

                cols, rows = np.nonzero(mask[:, :, 0])
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_X.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_X.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass
    
    def height_shift(self):
        
        for i in tqdm(range(len(self.df))):
            
            try:
                imgName = self.df.loc[i, "name"]
                img = cv2.imread(os.path.join(self.image_path_src, imgName))

                coor1 = (self.df.loc[i, "xmin"], self.df.loc[i, "ymin"])
                coor2 = (self.df.loc[i, "xmax"], self.df.loc[i, "ymax"])
                coor3 = (coor1[0], coor2[1])
                coor4 = (coor2[0], coor1[1])

                mask = np.zeros_like(img)
                polygons = np.array([[coor1, coor3, coor2, coor4]], np.int32)
                mask = cv2.fillPoly(mask, polygons, (255, 255, 255))

                # Vertical Width Shift
                percent_shift = np.random.uniform(-0.05, 0.05, 1)[0]
                T = np.float32([[1, 0, 0], [0, 1, percent_shift*img.shape[0]]])

                ## Image
                img = cv2.warpAffine(img, T, (img.shape[0], img.shape[1]))

                ## Mask
                mask = cv2.warpAffine(mask, T, (mask.shape[0], mask.shape[1]))

                cols, rows = np.nonzero(mask[:, :, 0])
                p1 = (np.min(rows), np.min(cols))
                p2 = (np.max(rows), np.max(cols))

                cv2.imwrite(os.path.join(self.image_path_des, imgName[:-4] + "_Y.png"), img)
                self.augment.loc[len(self.augment)] = [imgName[:-4] + "_Y.png", p1[0], p1[1], p2[0], p2[1]]
            
            except Exception as e:
                pass

In [ ]:
obj = Augmentation(data)
obj.flip_H()
obj.flip_V()
obj.rotate()
obj.shear()
obj.brightness()
obj.width_shift()
obj.height_shift()

In [ ]:
data = pd.concat([data, obj.augment], ignore_index=True)
data.to_csv("FinalData.csv", index=False, header=True)